In [4]:
import warnings
import os
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


X = np.load('./train_data/X_player.npy').astype(np.float32)[..., np.newaxis]
y = np.load('./train_data/y_player.npy').astype(np.float32)

y = y.reshape(y.shape[0], -1).astype(np.float32)

gpus = tf.config.experimental.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
print('\n\n Running on multiple GPUs ', [gpu.name for gpu in gpus])

with strategy.scope():
    model = load_model('/home/jinjinjara1022/OmokAI/models/best.h5')
    # (선택) 일부 레이어 동결
    # for layer in model.layers[:-2]:
    #     layer.trainable = False

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['acc']
    )

# 러닝레이트 스케줄러만 사용
callbacks = [
    ReduceLROnPlateau(monitor='acc', factor=0.2, patience=5, verbose=1)
]

# 학습
model.fit(
    x=X, y=y,
    batch_size=128,
    epochs=10,
    callbacks=callbacks,
    use_multiprocessing=True,
    workers=16
)

# 학습 완료 후 최종 모델 저장
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)
model.save(f'models/fine_tuned_{start_time}.h5')
print(f"▶ Fine-tuned model saved to models/fine_tuned_{start_time}.h5")

2025-05-14 14:11:23.332358: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...




 Running on multiple GPUs  []
Epoch 1/10


2025-05-14 14:11:23.767305: W tensorflow/core/framework/dataset.cc:956] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


1/1 [==============================] - 1s 1s/step - loss: 0.0271 - acc: 0.2821 - lr: 1.0000e-04
Epoch 2/10
1/1 [==============================] - 0s 147ms/step - loss: 0.0255 - acc: 0.2821 - lr: 1.0000e-04
Epoch 3/10
1/1 [==============================] - 0s 140ms/step - loss: 0.0241 - acc: 0.2821 - lr: 1.0000e-04
Epoch 4/10
1/1 [==============================] - 0s 152ms/step - loss: 0.0228 - acc: 0.2821 - lr: 1.0000e-04
Epoch 5/10
1/1 [==============================] - 0s 173ms/step - loss: 0.0215 - acc: 0.2821 - lr: 1.0000e-04
Epoch 6/10
1/1 [==============================] - 0s 168ms/step - loss: 0.0203 - acc: 0.3077 - lr: 1.0000e-04
Epoch 7/10
1/1 [==============================] - 0s 157ms/step - loss: 0.0191 - acc: 0.3077 - lr: 1.0000e-04
Epoch 8/10
1/1 [==============================] - 0s 145ms/step - loss: 0.0181 - acc: 0.3333 - lr: 1.0000e-04
Epoch 9/10
1/1 [==============================] - 0s 145ms/step - loss: 0.0171 - acc: 0.3333 - lr: 1.0000e-04
Epoch 10/10
1/1 [=======